In [ ]:
# from modelscope.utils.hf_util import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import torch
import json
from tqdm import tqdm
import pandas as pd

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/mnt/workspace/.cache/modelscope/hub/baichuan-inc/Baichuan2-7B-Chat",
                                          trust_remote_code=True)
device = torch.device('cuda:0')  # the device to load the model onto
model = AutoModelForCausalLM.from_pretrained("/mnt/workspace/.cache/modelscope/hub/baichuan-inc/Baichuan2-7B-Chat",
                                             device_map={"": device},
                                             torch_dtype="auto",
                                             trust_remote_code=True)
model.to(device)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    # do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40
)
hf = HuggingFacePipeline(pipeline=pipe)

### Baichuan2+KG

In [ ]:
# Read JSON file
file_path = 'aikps_output.json'  # Replace with your file path

# Open and read the JSON file
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Initialize an empty list to store results
results = []

# Use tqdm to display a progress bar
for item in tqdm(data, desc="Processing Progress"):
    question = 'If your smart device cannot understand what you say, will you feel frustrated?'
    context = '2. Humans have given AI the ability to understand human speech and complete tasks as required. (______)'
    
    template = """### This is a third-grade artificial intelligence question. 
        ### Please use the following context to enhance your answer.
        {context}
        ### Question: {question}
        ### Return the final result in JSON format as {{"Answer": ""}}.
    """
    
    prompt = PromptTemplate.from_template(template)
    
    # Assume hf is the predefined processing function
    chain = prompt | hf
    
    # Ensure variable names match those in the template
    result = chain.invoke({"context": context, "question": question})

    # Append context, question, and answer to the results list
    results.append([context, question, result])

# Convert the results list into a DataFrame
df = pd.DataFrame(results, columns=['Context', 'Question', 'Answer'])

# Write the DataFrame to a new Excel file
excel_path = 'Baichuan2+KG-output.xlsx'  # Replace with your desired output path
df.to_excel(excel_path, index=False)

### Baichuan2

In [ ]:
# Read JSON file
file_path = 'aikps_output.json'  # Replace with your file path

# Open and read the JSON file
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Initialize an empty list to store results
results = []

# Use tqdm to display a progress bar
for item in tqdm(data, desc="Processing Progress"):
    question = item[0]
    template = """### This is a third-grade Artificial Intelligence question. Please provide your answer.
        ### Question: {question}
        ### Return the final result in JSON format as {{"Answer": ""}}.
    """
    
    prompt = PromptTemplate.from_template(template)
    
    # Assume hf is the predefined processing function
    chain = prompt | hf
    
    # Ensure the variable name matches the placeholder in the template
    result = chain.invoke({"question": question})

    # Add question and answer to the results list
    results.append([question, result])

# Convert the results list into a DataFrame
df = pd.DataFrame(results, columns=['Question', 'Answer'])

# Write the DataFrame into a new Excel file
excel_path = 'Baichuan2-output.xlsx'  # Replace with your desired output file path
df.to_excel(excel_path, index=False)